In [1]:
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import os
from pylab import *
#from sklearn.neural_network import MLPClassifier


In [ ]:
##computes accuray 

def accuracy(Y,Y_hat):                
    sum1 = sum([1 for i in range(len(Y)) if Y[i] != Y_hat[i]])
    return sum1

In [ ]:
##coverts fetures in nin linear space
def define_Z(X,d):
    poly = PolynomialFeatures(d)
    Z = poly.fit_transform(X)
    return Z

In [ ]:
##taken from internet to shuffle two numpy array in Unision

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [ ]:
## indication function to return 1 and O depending on the class

def indicator(y,cls):
    if y==cls:
        return 1.0
    else:
        return 0.0


In [ ]:
## Softmax function returning a value between 0 and 1

def softmax(V,z,k):
    
    
    #print z.shape
    d = 0
    for v in V:
        
        #print v.shape
        
        d += exp(np.dot(np.transpose(v),z))
    n = exp(np.dot(np.transpose(V[k]),z))
    return (1.*n/d)
    
    

In [ ]:
##sigmoid function returnin value between zero and one
def sigmoid(w,x):
    
    return (1/(1 + exp(-np.dot(np.transpose(w),x))))

In [ ]:
##computes value of Y and Z on the basis of the estimated parameter

def compute_Z_Y(X,W,V):
    
    Z = []
    
    Y_hat = []
    
    for i in range(X.shape[0]):
        
        z = [1.0]
        
        y = []
        
        for w in W:
        
            z.append(sigmoid(w,X[i]))
        
        Z.append(z)          
            
        for k in range(V.shape[0]):
            
            #print k
            
            y.append(softmax(V,z,k))
            
            #print y
        
        Y_hat.append(y)
        
    #print Z
    #print Y_hat
              
    return np.array(Z),np.array(Y_hat)
        
  

In [ ]:
##computes likelihood for the predicted value of Y
def likelihood(X,Y,K,Y_hat):
    
    l = 0
    for i in range(X.shape[0]):
        for k in range(K):
            l += indicator(Y[i],k) * (log10(Y_hat[i][k]))
    l = - l
    return l
    

In [ ]:
##main training function 
##takes X Y H and learning rate as input and returns pewsicted value of Z and Y

def training(X,Y,cat,H,lr):
    
    W = np.zeros(shape=(H,X.shape[1]))
    
    W.fill(0.00)
    
    K = len(cat)
    
    V = np.zeros(shape=(K,H+1))
    
    V.fill(0.00)
    
    Z,Y_hat = compute_Z_Y(X,W,V)
    
    l_new = likelihood(X,Y,K,Y_hat)
    
    it = 0
    
    best = False
    
    while best == False: 
        
        l_pre = l_new
        
    
        for k in range(K):

            grad_V = 0

            for i in range(X.shape[0]):

                grad_V += (Y_hat[i][k] - indicator(Y[i],k))* Z[i]

            #print grad_V

            V[k] = V[k] - (lr*grad_V)



        for h in range(H):

            grad_W = 0

            for i in range(X.shape[0]):

                x = 0

                for k in range(K):

                    x += (Y_hat[i][k] - indicator(Y[i],k)) * V[k][h+1]


                    grad_W +=  x * Z[i][h+1] * (1.0 - Z[i][h+1]) * X[i]

                W[h] = W[h] - lr*grad_W


        Z,Y_hat = compute_Z_Y(X,W,V)

        it += 1

        l_new = likelihood(X,Y,K,Y_hat)
        
        print l_new
        
        if it==100 or abs(l_new-l_pre) < 0.5:
            
            best = True
        
        
    return V,W,Y_hat


In [ ]:
##computers the actual lable of Y (Eg: 0, 1 or 2)

def compute_Y_pred(Y_hat):
    Y_pred = []
    for y_hat in Y_hat:
        #print y_hat
        
        Y_pred.append(np.argmax(y_hat))
            
    return np.reshape(np.array(Y_pred),(len(Y_pred),1))

In [ ]:
##fetching the data from Sklearn package for classification only fro digit 0,1 and 2

mnist = fetch_mldata('MNIST original')

x, y = mnist.data / 255., mnist.target
X = []
Y = []
for i in range(len(y)):
    if y[i] == 0.0 or y[i] == 1.0 or y[i] == 2.0:
        X.append(x[i])
        Y.append(y[i])

X = np.array(X)

print X.shape
X = define_Z(X,1)
cat = (set(Y))
Y = np.reshape(np.array(Y),(len(Y),1))

X,Y = shuffle_in_unison(X, Y)


In [ ]:
### training and estimating parameters for 1000 rows
V,W,Y_hat = training(X[:1000],Y[:1000],cat,100,0.000001)
Y_pred = compute_Y_pred(Y_hat)

In [ ]:
print "Traning error", accuracy(Y[:1000],Y_pred)

In [ ]:
## predicting the value for next 500 observations
Z_test,Y_hat_test = compute_Z_Y(X[1000:1500],W,V)

In [ ]:
Y_pred_test = compute_Y_pred(Y_hat_test)

In [ ]:
print "testing error", accuracy(Y[1000:1500],Y_pred_test)

In [ ]:
################## actual MLP implementation######################################


mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4,
                    algorithm='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

mlp.fit(X[:1000], y_train[:1000])
print("Training set accuracy: %f" % mlp.score(X[:1000], y_train[:1000]))
print("Test set score: %f" % mlp.score(X[:500], y_test[:5000]))